In [1]:
import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
from mpi4py import MPI
import openfermion
from openfermion import FermionOperator

from openfermion import *


import warnings
warnings.filterwarnings("ignore", category=UserWarning)
#from qiskit_aer import AerSimulator
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

import scipy



comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

def generate_SQ_Operators():
    """
    n_orb is number of spatial orbitals assuming that spin orbitals are labelled
    0a,0b,1a,1b,2a,2b,3a,3b,....  -> 0,1,2,3,...
    """

    print(" Form singlet GSD operators")
    n_orb = 4 #I am passing it ----- Spatial orbitals 
    fermi_ops = []
    for p in range(0,n_orb):
        pa = 2*p
        pb = 2*p+1

        for q in range(p, n_orb):
            qa = 2*q
            qb = 2*q+1

            termA =  FermionOperator(((pa,1),(qa,0)))
            termA += FermionOperator(((pb,1),(qb,0)))

            termA -= hermitian_conjugated(termA)

            termA = normal_ordered(termA)

            #Normalize
            coeffA = 0
            for t in termA.terms:
                coeff_t = termA.terms[t]
                coeffA += coeff_t * coeff_t

            if termA.many_body_order() > 0:
                termA = termA/np.sqrt(coeffA)
                fermi_ops.append(termA)


    pq = -1
    for p in range(0,n_orb):
        pa = 2*p
        pb = 2*p+1

        for q in range(p,n_orb):
            qa = 2*q
            qb = 2*q+1

            pq += 1

            rs = -1
            for r in range(0, n_orb):
                ra = 2*r
                rb = 2*r+1

                for s in range(r, n_orb):
                    sa = 2*s
                    sb = 2*s+1

                    rs += 1

                    if(pq > rs):
                        continue


                    termA =  FermionOperator(((ra,1),(pa,0),(sa,1),(qa,0)), 2/np.sqrt(12))
                    termA += FermionOperator(((rb,1),(pb,0),(sb,1),(qb,0)), 2/np.sqrt(12))
                    termA += FermionOperator(((ra,1),(pa,0),(sb,1),(qb,0)), 1/np.sqrt(12))
                    termA += FermionOperator(((rb,1),(pb,0),(sa,1),(qa,0)), 1/np.sqrt(12))
                    termA += FermionOperator(((ra,1),(pb,0),(sb,1),(qa,0)), 1/np.sqrt(12))
                    termA += FermionOperator(((rb,1),(pa,0),(sa,1),(qb,0)), 1/np.sqrt(12))

                    termB =  FermionOperator(((ra,1),(pa,0),(sb,1),(qb,0)),  1/2.0)
                    termB += FermionOperator(((rb,1),(pb,0),(sa,1),(qa,0)),  1/2.0)
                    termB += FermionOperator(((ra,1),(pb,0),(sb,1),(qa,0)), -1/2.0)
                    termB += FermionOperator(((rb,1),(pa,0),(sa,1),(qb,0)), -1/2.0)

                    termA -= hermitian_conjugated(termA)
                    termB -= hermitian_conjugated(termB)

                    termA = normal_ordered(termA)
                    termB = normal_ordered(termB)

                    #Normalize
                    coeffA = 0
                    coeffB = 0
                    for t in termA.terms:
                        coeff_t = termA.terms[t]
                        coeffA += coeff_t * coeff_t
                    for t in termB.terms:
                        coeff_t = termB.terms[t]
                        coeffB += coeff_t * coeff_t


                    if termA.many_body_order() > 0:
                        termA = termA/np.sqrt(coeffA)
                        fermi_ops.append(termA)

                    if termB.many_body_order() > 0:
                        termB = termB/np.sqrt(coeffB)
                        fermi_ops.append(termB)

    n_ops = len(fermi_ops)
    print(" Number of operators: ", n_ops)
    return fermi_ops
# }}}

fermi_ops = generate_SQ_Operators()
x = [None] * len(fermi_ops)
print('Before loop, len of x', len(x))

for i in range(len(fermi_ops)):
    x[i] = qml.from_openfermion(fermi_ops[i])
print('Total operators after loop  are', len(x))


 Form singlet GSD operators
 Number of operators:  66
Before loop, len of x 66
Total operators after loop  are 66


In [2]:
print(x[2])

0.5 * a⁺(0) a(6)
+ 0.5 * a⁺(1) a(7)
+ -0.5 * a⁺(6) a(0)
+ -0.5 * a⁺(7) a(1)


In [ ]:

def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)

            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config=[]
    return list1
r = 1.8897259886 # 1 Angstrom in Bohr
distances = np.linspace(1.6*r, 2.35*r, 16)  # from 1 to bohr

# Setup other constants
symbols  = ['H', 'H', 'H', 'H']

excited_state_energies = []

# Define the molecule
for d in distances:   
    geometry = np.array([[0.0, 0.0,  0.0],
                        [0.0, 0.0, 1.5*r],
                        [d, 0.0, 0.0],
                        [d, 0.0, 1.5*r]
                        ], requires_grad = False)
    active_electrons = 4
    active_orbitals = 4
    charge = 0
   
    def gs_exact(symbols, coordinates, active_electrons, active_orbitals, charge, adapt_it):
        gr_energies = []
        ash_excitation = []
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
        #print('Number of qubits is', qubits)
        
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        #Calculation of HF state
        dev = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev)
        def circuit(hf_state, active_electrons, qubits, H):
            #print('Updated hf_state is', hf_state)  
            qml.BasisState(hf_state, wires=range(qubits))
            return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
        
        # Commutator calculation for HF state
        @qml.qnode(dev)
        def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
            qml.BasisState(k, wires=range(qubits))
            res = qml.commutator(H, w)   #Calculating the commutator
            return qml.expval(res)
        
        # Commutator calculation for other states except HF state
        @qml.qnode(dev)
        def commutator_1(H,w,k): #H is the Hamiltonian, w is the operator, k is the basis state
            qml.StatePrep(k, wires=range(qubits))
            res = qml.commutator(H, w) #Calculating the commutator
            return qml.expval(res)

        @qml.qnode(dev)
        def new_state(hf_state, params, ash_excitation, qubits):
            qml.BasisState(hf_state, wires=range(qubits))
            for i in range(len(ash_excitation)):
                #print(".....", type(ash_excitation))
                #print('Params now are', params[i])
                s = qml.jordan_wigner(x[ash_excitation[i]])
                qml.exp((s * params[i]/2), num_steps=1) 
            return qml.state()
        
        @qml.qnode(dev)
        def ash(hf_state, params, ash_excitation, qubits):
            qml.BasisState(hf_state, wires=range(qubits))
            for i in range(len(ash_excitation)):
                #print('Operator dealing now is', ash_excitation[i])
                #print('Params now are', params[i])
                s = qml.jordan_wigner(x[ash_excitation[i]])
                qml.exp((s * params[i]/2), num_steps=1) 
            return qml.expval(H)


        def cost(params):
            energy = ash(hf_state, params, ash_excitation, qubits)
            return energy


        print('HF state is', circuit(hf_state, active_electrons, qubits, H))
        
        states = [hf_state]
        params = np.zeros(len(ash_excitation), requires_grad=True) 

        null_state = np.zeros(qubits,int)
        #print('Null state is', null_state)

        for j in range(adapt_it):
            #print('The adapt iteration now is', j)  #Adapt iteration
            max_value = float('-inf')
            #max_operator = None
            k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
            #for i in operator_pool:
            for i in range(len(x)):
                #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(x[i])  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

            ash_excitation.append(max_operator) #Appending the excitations to the ash_excitation
            #print('Ash excitation after adding operator', ash_excitation)
            params = np.append(params, 0.0)  #Parameters initialization

            #Energy calculation
            result = minimize(cost, params, method='powell', tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

            #print("Final updated parameters:", result.x)
            #print("Final cost:", result.fun)

            params= (result.x)
            gr_energies.append(result.fun)


            ostate = new_state(hf_state, params, ash_excitation, qubits)
            #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
            #gs_state = ostate
            states.append(ostate)
        print("gr_energies:", gr_energies[-1])
        return params, ash_excitation

    def qsc_eom(symbols, coordinates, active_electrons, active_orbitals, charge, params, ash_excitation, shots=0):
    # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method='pyscf', active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        hf_state = qml.qchem.hf_state(active_electrons, qubits)

        wires=range(qubits)
        null_state = np.zeros(qubits,int)
        list1 = inite(active_electrons,qubits)
        if shots==0:
            dev = qml.device("lightning.qubit", wires=qubits)
        else:
            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
        
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, hf_state, ash_excitation):
            for w in occ:
                qml.X(wires=w)
            for i in range(len(ash_excitation)):
                s = qml.jordan_wigner(x[ash_excitation[i]])
                qml.exp((s * params[i]/2), num_steps=1) 
            return qml.expval(H)
        
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2, wires, hf_state, ash_excitation):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for i in range(len(ash_excitation)):
                s = qml.jordan_wigner(x[ash_excitation[i]])
                qml.exp((s * params[i]/2), num_steps=1) 
            return qml.expval(H)

        #final M matrix
        mat_size = len(list1)
        #print("Size of Matrix M is", mat_size)


        block_size = mat_size/size
        lo_limit = int(rank * block_size)
        up_limit = int((rank+1) * block_size)
        if rank == size-1:
            up_limit = mat_size

        #print ("Rank: ", rank, " Low limit: ", lo_limit, " Upper Limit: ", up_limit)

        M = np.zeros((mat_size, mat_size))
        M_1d_size = int(mat_size * (mat_size + 1)/2)
        #eig = np.zeros(mat_size)
        Mdiag = np.zeros(mat_size)
        Mdiag_local = np.zeros(mat_size)
        M_local = np.zeros((mat_size, mat_size))
        for i in range(lo_limit, up_limit):
            Mdiag_local[i] = circuit_d(params, list1[i], wires, null_state, ash_excitation)
        comm.Reduce(Mdiag_local, Mdiag, MPI.SUM, 0)
        comm.Bcast(Mdiag, root=0)
        #print("Rank: ", rank, " diagonal parts done", "Mdiag_local: ", Mdiag_local)
        #print("Rank: ", rank, " diagonal parts done", "Mdiag: ", Mdiag)
        def idx(n):
            id_i = np.ceil((-1 + np.sqrt(1 + 4 * 2 * n))/2)
            id_j = n - (id_i-1) * (id_i)/2
            return int(id_i-1), int(id_j-1)

        for ii in range(rank, M_1d_size, size):
            c = ii + 1

            idi, idj = idx(c)

            if idi==idj:
                Mtmp = Mdiag[idi]
            else:
                #print(idi, idj)
                Mtmp = circuit_od(params, list1[idi], list1[idj], wires, null_state, ash_excitation) - Mdiag[idi]/2.0 - Mdiag[idj]/2.0

            M_local[idi, idj] = Mtmp
            M_local[idj, idi] = Mtmp

        #print("Rank: ", rank, " off diagonal terms done")
        comm.Reduce(M_local, M, MPI.SUM, 0)
        comm.Bcast(M, root=0)
    #   print('Rank:', rank, 'Matrix: ', M[0:10])
        #for i in range(mat_size):
        #    M[i, i] = Mdiag[i]

        #comm.Barrier

        #t3 = time()
        eig, evec=np.linalg.eig(M)
        #t4 = time()
        #print("Time taken in diagonalization:", t4-t3)

        return np.sort(eig), evec
    print(f"Computing for bond length: {d / r:.2f} Å")
    params, ash_excitation = gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, adapt_it=3)

    eig, evec = qsc_eom(symbols, geometry, active_electrons, active_orbitals, charge, params, ash_excitation, shots=0)
    #np.save('eigenvalues.npy', eig)
    #np.save('eigenvectors.npy', evec)
    excited_state_energies.append(eig)
    print("qscEOM", excited_state_energies)


Computing for bond length: 1.60 Å
HF state is -1.7408402634666045


/Users/prince.kwao/miniconda3/envs/prince/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


gr_energies: -1.9011003480025006
qscEOM [tensor([-1.9484036 , -1.89438047, -1.82299428, -1.80568569, -1.79974341,
        -1.72365376, -1.61002887, -1.44409347, -1.41992567, -1.41840158,
        -1.4114045 , -1.36857138, -1.33284629, -1.3282944 , -1.28768248,
        -1.25414921, -1.23171359, -1.17992173, -1.14218551, -1.14111091,
        -1.13744108, -1.04077387, -0.96941389, -0.96389227, -0.75718974,
        -0.47157947, -0.45149258], requires_grad=True)]
Computing for bond length: 1.65 Å
HF state is -1.7517549602880862
gr_energies: -1.9103248820412062
qscEOM [tensor([-1.9484036 , -1.89438047, -1.82299428, -1.80568569, -1.79974341,
        -1.72365376, -1.61002887, -1.44409347, -1.41992567, -1.41840158,
        -1.4114045 , -1.36857138, -1.33284629, -1.3282944 , -1.28768248,
        -1.25414921, -1.23171359, -1.17992173, -1.14218551, -1.14111091,
        -1.13744108, -1.04077387, -0.96941389, -0.96389227, -0.75718974,
        -0.47157947, -0.45149258], requires_grad=True), tensor([-1

In [4]:
qscEOM_gr = np.array(excited_state_energies)
qs =qscEOM_gr[:,0]
print(qs)

[-1.9484036  -1.95275161 -1.95734791 -1.96182915 -1.96601379 -1.96982331
 -1.97323532 -1.97625755 -1.97891325 -1.98123279 -1.9832491  -1.9849949
 -1.9865015  -1.98779786 -1.98891036 -1.98986273]
